In [9]:
import pandas as pd
import os
import json
import umap
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import tensorflow_hub as hub
import numpy as np
from detoxify import Detoxify
import tweetnlp

In [10]:
DATA_FPATH = '../react-app/api/data/moderation/'

In [11]:
def getModelPredsHelper(allExamples, model, return_string=False):
    if type(model) == fasttext.FastText._FastText: 
        exampleList = [x.replace('\n', ' ') for x in allExamples]
        preds = model.predict(exampleList)
        if task == 'moderation':
            preds_int = np.array([1 if 'positive' in p[0] else 0 for p in preds[0]])
        else: 
            preds_int = np.array([1 if '2' in p[0] else 0 for p in preds[0]])

        preds_prob = np.array([p[0] for p in preds[1]])
        probs = np.array([1 - p if preds_int[i] == 0 else p for (i,p) in enumerate(preds_prob)])

    elif type(model) == Detoxify:
        probs = model.predict(allExamples)['toxicity']

    elif type(model) == tweetnlp.model_text_classification.model.Offensive:
        probs_dict = model.predict(allExamples)
        probs = [p['probability'] if p['label']=='offensive' else 1 - p['probability'] for p in probs_dict]

    if return_string: 
        return [str(np.round(p,3)) for p in probs]
    else: 
        return np.round(probs,3)

In [12]:
def check_valid_text(comment_text):
    bot_terms = ["I'm a bot", "I am a bot"]
    no_text_terms = ["[removed]", "[deleted]"]
    automod_terms = ["If you feel this was done in error, or would like further clarification, please don't hesitate to"]
    link_terms = ["http", "https", ".com"]
    
    for t in bot_terms + no_text_terms + automod_terms + link_terms: 
        if t in comment_text:
            return False
    return True

In [13]:
moderated_df = pd.read_csv(DATA_FPATH + 'reddit-removal-log.csv')

In [14]:
moderated_df['subreddit'] = moderated_df.subreddit.str.lower()

In [15]:
all_dfs = {}

In [16]:
for sub in os.listdir(DATA_FPATH): 
    if sub == '.DS_Store' or sub == 'reddit-removal-log.csv': continue
    
    # moderated comments from reddit-removal-log
    moderated_comments = moderated_df[moderated_df.subreddit == sub]
    moderated_comments['moderated'] = 1
    moderated_comments = moderated_comments[moderated_comments.body.apply(check_valid_text)]
    moderated_comments = moderated_comments.drop_duplicates(subset="body")
    moderated_comments = moderated_comments.sample(5000)
    
    # unmoderated comments from scraped data 
    unmoderated_comments = []
    for idx in range(201):
        with open(DATA_FPATH + sub + "/" + str(idx) + ".data", "r") as f:
            comment_obj = json.load(f)["data"]
            comment_list = [(item["body"], sub, 0) for item in comment_obj if item["author"] != "AutoModerator"]
            unmoderated_comments.extend(comment_list)
    unmoderated_comments = pd.DataFrame(unmoderated_comments, columns=["body", "subreddit", "moderated"])
    unmoderated_comments = unmoderated_comments[unmoderated_comments.body.apply(check_valid_text)]
    unmoderated_comments = unmoderated_comments
    unmoderated_comments = unmoderated_comments.drop_duplicates(subset="body")
    unmoderated_comments = unmoderated_comments.sample(10000)
    
    # Combine
    all_comments = pd.concat([moderated_comments, unmoderated_comments], ignore_index=True)
    all_dfs[sub] = all_comments

/var/folders/mr/c6zcp4tj0nq9fj8sdpnnsfj80000gn/T/ipykernel_16500/3267666709.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moderated_comments['moderated'] = 1


In [17]:
embed = hub.load("/Users/harinisuresh/Dev/tf_hub/universal-sentence-encoder_4/")

2022-08-18 22:07:19.953979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
model_1 = Detoxify('original')
model_2 = tweetnlp.load('offensive')

In [16]:
for sub in ['tifu']: 
    if sub == '.DS_Store': continue
    sub_df = all_dfs[sub]
    sub_comments = sub_df.body
    vecs = np.array(embed(sub_comments))
    umap_reducer = umap.UMAP()
    umap_embeddings = umap_reducer.fit_transform(vecs)
    pca_reducer = PCA(n_components=2)
    pca_embeddings = pca_reducer.fit_transform(vecs)
    tsne_reducer = TSNE(n_components=2)
    tsne_embeddings = tsne_reducer.fit_transform(vecs)
    detoxify_preds = model_1.predict(list(sub_comments))
    tweetnlp_preds = model_2.predict(list(sub_comments))
    new_df = pd.DataFrame({
        'comment_id': range(len(sub_comments)),
        'comment': sub_comments,
        'label': sub_df.moderated,
        'umap_x': umap_embeddings[:,0],
        'umap_y': umap_embeddings[:,1],
        'pca_x': pca_embeddings[:,0],
        'pca_y': pca_embeddings[:,1],
        'tsne_x': tsne_embeddings[:,0],
        'tsne_y': tsne_embeddings[:,1],
        'detoxify_preds': detoxify_preds,
        'tweetnlp_preds': tweetnlp_preds
    })
    all_dfs[sub] = new_df

/Users/harinisuresh/.virtualenvs/new_test_cases/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/harinisuresh/.virtualenvs/new_test_cases/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [ ]:
preds = model_1.predict(list(all_dfs['tifu'].body.sample(300)))

In [ ]:
df.to_csv(DATA_FPATH + sub + '/projection_df', index=False)

In [ ]:
np.save(DATA_FPATH + sub + '/embs.npy')